# Fine-tune Llama 2 on iSarcasmEval dataset
### This notebook is  inspired by [Fine-tune Llama 2 for sentiment analysis](https://www.kaggle.com/code/lucamassaron/fine-tune-llama-2-for-sentiment-analysis/notebook)  by **Luca Massaron** and [Fine-Tuning LLaMA 2](https://www.datacamp.com/tutorial/fine-tuning-llama-2)

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
!pip install -q -U "torch==2.1.2" tensorboard
!pip install -q -U "transformers==4.36.2" "datasets==2.16.1" "accelerate==0.26.1" "bitsandbytes==0.42.0"
!pip install -q -U git+https://github.com/huggingface/trl@a3c5b7178ac4f65569975efadc97db2f3749c65e
!pip install -q -U git+https://github.com/huggingface/peft@4a1559582281fc3c9283892caea8ccef1d6f5a4f
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TOKENIZERS_PARALLELISM"] = "false"
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
# import os
from tqdm import tqdm
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import Dataset
from peft import LoraConfig, PeftConfig
from trl import SFTTrainer
from trl import setup_chat_format
from transformers import (AutoModelForCausalLM,
                          AutoTokenizer,
                          BitsAndBytesConfig,
                          TrainingArguments,
                          pipeline,
                          logging)
from sklearn.metrics import (accuracy_score,
                             classification_report,
                             confusion_matrix)
from sklearn.model_selection import train_test_split

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 96.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.8/209.8 MB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 MB 19.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires tensorboard<2.16,>=2.15, but you have tensorboard 2.16.2 which is incompatible.
torchaudio 2.2.1+cu121 requires torch==2.2.1, but you have torch 2.1.2 which is incompatible.
torchtext 0.17.1 requires torch==2.2.1, but you have torch 2.1.2 which is incompatible.
torchvision 0.17.1+cu121 requires torch==2.2.1, but you have torch 2.1.2 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import numpy as np
import pandas as pd
import re

## Preparing Data

In [ ]:
!git clone https://github.com/iabufarha/iSarcasmEval.git

# External Data
!git clone https://github.com/headacheboy/data-of-multimodal-sarcasm-detection.git

Cloning into 'iSarcasmEval'...
remote: Enumerating objects: 72, done.
remote: Counting objects: 100% (72/72), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 72 (delta 25), reused 32 (delta 6), pack-reused 0
Receiving objects: 100% (72/72), 535.16 KiB | 5.30 MiB/s, done.
Resolving deltas: 100% (25/25), done.
Cloning into 'data-of-multimodal-sarcasm-detection'...
remote: Enumerating objects: 68, done.
remote: Total 68 (delta 0), reused 0 (delta 0), pack-reused 68
Receiving objects: 100% (68/68), 11.49 MiB | 20.93 MiB/s, done.
Resolving deltas: 100% (17/17), done.


# replace abbreviation

In [ ]:
#https://stackoverflow.com/questions/57503786/python-replace-abbreviation-in-text
replacers = {'dm': 'direct message',
 'thx': 'thanks',
 'dming': 'direct messaging',
 'dmed': 'direct messaged',
 'plz': 'please',
 'u': 'you',
 'asap': 'as soon as possible',
 '...': '',
 '. . .': '',
 'r': 'are',
 'U':'You',
 'idk':"i don't know",
 'omg':'oh my god',
 'sry':'sorry',
  'fb':'facebook',
 }

replace_abb = lambda x: ' '.join([replacers.get(e, e) for e in x])


# replace USERNAME

In [ ]:
replace_username = lambda x: re.sub('@[^\s]+','<user>',x)

# replace_hashtag = lambda x: re.sub('#[^\s]+','<TAG>',x)

replace_link = lambda x: re.sub(r'http?s?://\S+', '<url>', x)

In [ ]:
def process(text):
    text = replace_abb(str(text).split())
    text = replace_username(text)
    # text = replace_hashtag(text)
    text = replace_link(text)
    return text

In [ ]:
import re
import random
random.seed(42)

In [ ]:
text_file = open("/content/data-of-multimodal-sarcasm-detection/text/train.txt", "r")
lines = text_file.readlines()

# string representation of list to list using ast.literal_eval()
import ast

data_list = []

one = 0
zero = 1

# Converting string to list
for i,line in enumerate(lines):
  res = ast.literal_eval(line)
  data_list.append(res)
  if i == 10000:
    break


X_2 = pd.DataFrame(data_list)

In [ ]:
X_2.rename(columns = { '1': 'tweet', '2': 'sarcastic' }, inplace=True)

In [ ]:
X_3 = X_2.rename(columns = {1: 'tweet', 2: 'sarcastic' })
X_3 = X_3[['tweet','sarcastic']]

In [ ]:
X_3['tweet'] = X_3['tweet'].apply(process)

In [ ]:
X_3

,tweet,sarcastic
0,most # funny quotes : 21 snarky and # funny qu...,1
1,spurs # creativethinking ! <url>,1
2,<user> thanks for showing up for our appointme...,1
3,only a hardcore fan of sir jonny sins will get...,1
4,haha . # lol,1
...,...,...
9996,"barely a teenager , cj collins is now part of ...",0
9997,excited to have the opportunity to compete at ...,0
9998,i 'm in a <user> campaign and that 's wild to ...,0
9999,the boy found this view particularly enticing ...,0


In [ ]:
filename ='/content/iSarcasmEval/train/train.En.csv'
X = pd.read_csv(filename,
                 usecols=["tweet", "sarcastic"],
                 encoding="utf-8", encoding_errors="replace")



In [ ]:
X['tweet'] = X['tweet'].apply(process)

In [ ]:
X

,tweet,sarcastic
0,The only thing I got from college is a caffein...,1
1,I love it when professors draw a big question ...,1
2,Remember the hundred emails from companies whe...,1
3,Today my pop-pop told me I was not “forced” to...,1
4,"<user> <user> <user> I did too, and I also rep...",1
...,...,...
3463,The population spike in Chicago in 9 months is...,0
3464,You'd think in the second to last English clas...,0
3465,I’m finally surfacing after a holiday to Scotl...,0
3466,Couldn't be prouder today. Well done to every ...,0


In [ ]:
X = pd.concat([X, X_3], ignore_index=True,axis=0)

In [ ]:
def generate_prompt(data_point):
    return f"""
            Determine the whether the tweet enclosed in square brackets is sarcastic or non-sarcastic,
            and return the answer as the corresponding label "1" for sarcastic or "0" for non-sarcastic.

            Some words like "really", "never", "actually", etc. can be considered a symbol of sarcasm and express some contradictory and criticized attitudes.

            We also notice that some misspelled words (e.g., "so"->"soooo", "love"->"looove", "sure"->"sureeee") and capitalized words (not located at the beginning of a sentence) can sometimes exaggerate the emotional expression.


            [{data_point["tweet"]}] = {data_point["sarcastic"]}
            """.strip()

X = pd.DataFrame(X.apply(generate_prompt, axis=1),
                       columns=["text"])

X = X.sample(frac=1)

train_size = int(len(X)*0.8)
val_size = len(X)-train_size

X_train = X[:train_size]
X_val = X[train_size:]

In [ ]:
X_train

,text
7008,Determine the whether the tweet enclosed in sq...
5515,Determine the whether the tweet enclosed in sq...
11504,Determine the whether the tweet enclosed in sq...
2044,Determine the whether the tweet enclosed in sq...
6914,Determine the whether the tweet enclosed in sq...
...,...
5178,Determine the whether the tweet enclosed in sq...
1266,Determine the whether the tweet enclosed in sq...
10172,Determine the whether the tweet enclosed in sq...
1813,Determine the whether the tweet enclosed in sq...


In [ ]:
X_val

,text
10794,Determine the whether the tweet enclosed in sq...
9424,Determine the whether the tweet enclosed in sq...
11009,Determine the whether the tweet enclosed in sq...
8997,Determine the whether the tweet enclosed in sq...
10747,Determine the whether the tweet enclosed in sq...
...,...
7753,Determine the whether the tweet enclosed in sq...
11940,Determine the whether the tweet enclosed in sq...
3492,Determine the whether the tweet enclosed in sq...
7696,Determine the whether the tweet enclosed in sq...


In [ ]:
test_filename ='/content/iSarcasmEval/test/task_A_En_test.csv'

X_test = pd.read_csv(test_filename,
                 usecols=["text", "sarcastic"],
                #  names = ["tweet", "sarcastic"],
                 encoding="utf-8", encoding_errors="replace")

X_test = X_test.rename(columns={'text':'tweet'})

X_test = X_test.sample(frac=1)

def generate_test_prompt(data_point):
    return f"""
            Determine whether the tweet enclosed in square brackets is sarcastic or non-sarcastic,
            and return the answer as the corresponding label "1" for sarcastic or "0" for non-sarcastic.

            Some words like "really", "never", "actually", etc. can be considered a symbol of sarcasm and express some contradictory and criticized attitudes.

            We also notice that some misspelled words (e.g., "so"->"soooo", "love"->"looove", "sure"->"sureeee") and capitalized words (not located at the beginning of a sentence) can sometimes exaggerate the emotional expression.

            [{data_point["tweet"]}] = """.strip()


y_test = X_test['sarcastic']
X_test = pd.DataFrame(X_test.apply(generate_test_prompt, axis=1), columns=["text"])

In [ ]:
X_test

,text
108,Determine the whether the tweet enclosed in sq...
1235,Determine the whether the tweet enclosed in sq...
490,Determine the whether the tweet enclosed in sq...
159,Determine the whether the tweet enclosed in sq...
143,Determine the whether the tweet enclosed in sq...
...,...
324,Determine the whether the tweet enclosed in sq...
440,Determine the whether the tweet enclosed in sq...
177,Determine the whether the tweet enclosed in sq...
566,Determine the whether the tweet enclosed in sq...


## Functions

In [ ]:
def evaluate(y_true, y_pred):
    # Calculate accuracy
    accuracy = accuracy_score(y_true=y_true, y_pred=y_pred)
    print(f'Accuracy: {accuracy:.3f}')

    # Generate accuracy report
    unique_labels = set(y_true)  # Get unique labels

    for label in unique_labels:
        label_indices = [i for i in range(len(y_true))
                         if y_true[i] == label]
        label_y_true = [y_true[i] for i in label_indices]
        label_y_pred = [y_pred[i] for i in label_indices]
        accuracy = accuracy_score(label_y_true, label_y_pred)
        print(f'Accuracy for label {label}: {accuracy:.3f}')

    # Generate classification report
    class_report = classification_report(y_true=y_true, y_pred=y_pred)
    print('\nClassification Report:')
    print(class_report)

    # Generate confusion matrix
    conf_matrix = confusion_matrix(y_true=y_true, y_pred=y_pred, labels=[0, 1])
    print('\nConfusion Matrix:')
    print(conf_matrix)

In [ ]:
compute_dtype = getattr(torch, "float16")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=True,
)


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"working on {device}")

working on cuda:0


In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf",trust_remote_code=True,)
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                             device_map=device,
                                              torch_dtype=compute_dtype,
                                              quantization_config=bnb_config,)

model.config.use_cache = False
model.config.pretraining_tp = 1
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

model, tokenizer = setup_chat_format(model, tokenizer)

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [ ]:
def predict(test, model, tokenizer):
    y_pred = []
    for i in tqdm(range(len(X_test))):
        prompt = X_test.iloc[i]["text"]
        pipe = pipeline(task="text-generation",
                        model=model,
                        tokenizer=tokenizer,
                        max_new_tokens=2
                       )
        result = pipe(prompt)
        answer = result[0]['generated_text'].split("=")[-1]
        if "1" in answer:
            y_pred.append(1)
        elif "0" in answer:
            y_pred.append(0)
        else:
            y_pred.append(2)
    return y_pred

## Fine-tuning

In [ ]:
train_data = Dataset.from_pandas(X_train)
eval_data = Dataset.from_pandas(X_val)

In [ ]:
output_dir="trained_weigths"

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

training_arguments = TrainingArguments(
    output_dir=output_dir,                    # directory to save and repository id
    num_train_epochs=4,                       # number of training epochs
    per_device_train_batch_size=4,            # batch size per device during training
    gradient_accumulation_steps=1,            # number of steps before performing a backward/update pass
    gradient_checkpointing=True,              # use gradient checkpointing to save memory
    optim="paged_adamw_32bit",
    save_steps=0,
    logging_steps=25,                         # log every 10 steps
    learning_rate=2e-4,                       # learning rate, based on QLoRA paper
    weight_decay=0.001,
    fp16=True,
    bf16=False,
    max_grad_norm=0.3,                        # max gradient norm based on QLoRA paper
    max_steps=-1,
    warmup_ratio=0.03,                        # warmup ratio based on QLoRA paper
    group_by_length=True,
    lr_scheduler_type="cosine",               # use cosine learning rate scheduler
    report_to="tensorboard",                  # report metrics to tensorboard
    evaluation_strategy="epoch"               # save checkpoint every epoch
)



In [ ]:
trainer = SFTTrainer(
    model=model,
    args=training_arguments,
    train_dataset=train_data,
    eval_dataset=eval_data,
    peft_config=peft_config,
    dataset_text_field="text",
    tokenizer=tokenizer,
    max_seq_length=1024,
    packing=False,
    dataset_kwargs={
        "add_special_tokens": False,
        "append_concat_token": False,
    }
)

Map:   0%|          | 0/10775 [00:00<?, ? examples/s]

Map:   0%|          | 0/2694 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,0.450400,0.392994
2,0.395400,0.385113
3,0.309000,0.386993
4,0.252200,0.394037


TrainOutput(global_step=10776, training_loss=0.3670381877616977, metrics={'train_runtime': 11616.3223, 'train_samples_per_second': 3.71, 'train_steps_per_second': 0.928, 'total_flos': 3.471159189848064e+17, 'train_loss': 0.3670381877616977, 'epoch': 4.0})

## Evaluate

In [ ]:
y_pred = predict(X_test, model, tokenizer)

100%|██████████| 1400/1400 [09:19<00:00,  2.50it/s]


In [ ]:
evaluate(y_test.tolist(), y_pred)

Accuracy: 0.841
Accuracy for label 0: 0.852
Accuracy for label 1: 0.775

Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.85      0.90      1200
           1       0.47      0.78      0.58       200

    accuracy                           0.84      1400
   macro avg       0.71      0.81      0.74      1400
weighted avg       0.89      0.84      0.86      1400


Confusion Matrix:
[[1022  178]
 [  45  155]]
